# Preprocess Raw Data and Build Dataset

In [215]:
import os
import pandas as pd
import numpy as np

# Load Data

In [216]:
data_folder = os.path.join('..', 'data')

raw_data_folder = os.path.join(data_folder, 'raw_data')
intermediate_data_folder = os.path.join(data_folder, 'intermediate_data')

In [217]:
datasets_dict = {}

datasets_dict['TORTOSA'] = {}
datasets_dict['GUIAMETS'] = {}
datasets_dict['MEQUINENZA'] = {}
datasets_dict['XERTA'] = {}

In [218]:
for file in os.listdir(raw_data_folder):
    location = file.split('_')[0]
    feature_name = '_'.join(file.split('_')[1:-2])
    if file.endswith('.csv'):
         datasets_dict[location][feature_name] = pd.read_csv(
            filepath_or_buffer=os.path.join(raw_data_folder, file),
            sep=';',
            decimal=',',
            date_format='%Y-%m-%d %H:%M:%S',
            header=0,
            encoding='utf-8',
        )
    elif file.endswith('.xlsx'):
        datasets_dict[location][feature_name] = pd.read_excel(
            os.path.join(raw_data_folder, file),
            date_format='%Y-%m-%d %H:%M:%S',
            header=0,
        )

In [219]:
tortosa_dfs = datasets_dict['TORTOSA']
guiamets_dfs = datasets_dict['GUIAMETS']
mequinenza_dfs = datasets_dict['MEQUINENZA']
xerta_dfs = datasets_dict['XERTA']

# Tortosa Preprocessing

In [220]:
tortosa_dfs.keys()

dict_keys(['turbidity', 'cumulated_rainfall_24h', 'flowriver', 'water_temperature', 'conductivity', 'watertemperature'])

In [221]:
# The water temperature has two datasets, but the excel one has no missing values
tortosa_dfs['watertemperature'].isna().sum() / tortosa_dfs['watertemperature'].shape[0]

Fecha       0.0
Promedio    0.0
Mínimo      0.0
Máximo      0.0
Nºdat       0.0
NºOK        0.0
dtype: float64

In [222]:
tortosa_dfs['water_temperature'].isna().sum() / tortosa_dfs['water_temperature'].shape[0]

M�ximo          0.014815
M�nimo          0.014815
Media           0.014815
Fecha m�ximo    0.014815
Fecha m�nimo    0.014815
fecha           0.000000
dtype: float64

In [223]:
tortosa_dfs.pop('water_temperature')

,M�ximo,M�nimo,Media,Fecha m�ximo,Fecha m�nimo,fecha
0,28.0,26.9,27.8,2012-08-29 16:15:00,2012-08-29 12:45:00,2012-08-29 00:00:00
1,27.6,26.2,26.7,2012-08-30 00:00:00,2012-08-30 23:45:00,2012-08-30 00:00:00
2,26.1,24.7,25.3,2012-08-31 00:00:00,2012-08-31 09:45:00,2012-08-31 00:00:00
3,24.8,23.5,23.9,2012-09-01 00:00:00,2012-09-01 11:45:00,2012-09-01 00:00:00
4,24.2,22.6,23.3,2012-09-02 18:00:00,2012-09-02 12:00:00,2012-09-02 00:00:00
...,...,...,...,...,...,...
4045,26.3,24.9,25.5,2023-09-26 17:30:00,2023-09-26 07:15:00,2023-09-26 00:00:00
4046,26.9,25.5,26.1,2023-09-27 18:30:00,2023-09-27 07:30:00,2023-09-27 00:00:00
4047,27.2,25.7,26.3,2023-09-28 18:30:00,2023-09-28 09:00:00,2023-09-28 00:00:00
4048,27.4,25.9,26.5,2023-09-29 18:15:00,2023-09-29 09:00:00,2023-09-29 00:00:00


In [224]:
# Check cumulated rainfall data since it is the only csv file
tortosa_dfs['cumulated_rainfall_24h']

,Acumulado,Fecha acumulado,M�ximo,Fecha m�ximo,fecha
0,0.0,2004-07-04 00:00:00,NaN,NaN,2004-07-04 00:00:00
1,0.8,2004-07-05 00:00:00,NaN,NaN,2004-07-05 00:00:00
2,2.4,2004-07-06 00:00:00,NaN,NaN,2004-07-06 00:00:00
3,0.8,2004-07-07 00:00:00,NaN,NaN,2004-07-07 00:00:00
4,0.0,2004-07-08 00:00:00,NaN,NaN,2004-07-08 00:00:00
...,...,...,...,...,...
7023,0.0,2023-09-26 00:00:00,NaN,NaN,2023-09-26 00:00:00
7024,0.0,2023-09-27 00:00:00,NaN,NaN,2023-09-27 00:00:00
7025,0.0,2023-09-28 00:00:00,NaN,NaN,2023-09-28 00:00:00
7026,0.0,2023-09-29 00:00:00,NaN,NaN,2023-09-29 00:00:00


In [225]:
# fecha column is the one to take into account since 
# it is equal to the Fecha acumulado column in the same dataframe
# but it has no missing values
mask = tortosa_dfs['cumulated_rainfall_24h']['Fecha acumulado'] == tortosa_dfs['cumulated_rainfall_24h']['fecha']
tortosa_dfs['cumulated_rainfall_24h'][mask == False]

,Acumulado,Fecha acumulado,M�ximo,Fecha m�ximo,fecha
69,NaN,NaN,NaN,NaN,2004-09-11 00:00:00
70,NaN,NaN,NaN,NaN,2004-09-12 00:00:00
71,NaN,NaN,NaN,NaN,2004-09-13 00:00:00
72,NaN,NaN,NaN,NaN,2004-09-14 00:00:00
73,NaN,NaN,NaN,NaN,2004-09-15 00:00:00
74,NaN,NaN,NaN,NaN,2004-09-16 00:00:00
75,NaN,NaN,NaN,NaN,2004-09-17 00:00:00
76,NaN,NaN,NaN,NaN,2004-09-18 00:00:00
77,NaN,NaN,NaN,NaN,2004-09-19 00:00:00
78,NaN,NaN,NaN,NaN,2004-09-20 00:00:00


In [226]:
tortosa_dfs['cumulated_rainfall_24h'].isna().sum() / tortosa_dfs['cumulated_rainfall_24h'].shape[0]

Acumulado          0.004269
Fecha acumulado    0.004269
M�ximo             1.000000
Fecha m�ximo       1.000000
fecha              0.000000
dtype: float64

In [227]:
tortosa_dfs['cumulated_rainfall_24h'] = tortosa_dfs['cumulated_rainfall_24h'][['fecha', 'Acumulado']].rename(columns={'fecha': 'DateTime', 'Acumulado': 'Average'})

In [228]:
tortosa_dfs['conductivity'].isna().sum() / tortosa_dfs['conductivity'].shape[0]

Fecha       0.0
Promedio    0.0
Mínimo      0.0
Máximo      0.0
Nºdat       0.0
NºOK        0.0
dtype: float64

In [229]:
tortosa_dfs['flowriver'].isna().sum() / tortosa_dfs['flowriver'].shape[0]

Fecha       0.0
Promedio    0.0
Mínimo      0.0
Máximo      0.0
Nºdat       0.0
NºOK        0.0
dtype: float64

In [230]:
tortosa_dfs['turbidity'].isna().sum() / tortosa_dfs['turbidity'].shape[0]

Fecha       0.0
Promedio    0.0
Mínimo      0.0
Máximo      0.0
Nºdat       0.0
NºOK        0.0
dtype: float64

In [231]:
for feature, df in tortosa_dfs.items():
    if feature != 'cumulated_rainfall_24h':
        df.rename(
            columns={
                'Fecha': 'DateTime',
                'Promedio': 'Average',
                'Máximo': 'Maximum',
                'Mínimo': 'Minimum'
            },
            inplace=True
        )

In [232]:
for df in tortosa_dfs.values():
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df[df.columns.difference(['DateTime'])] = df[df.columns.difference(['DateTime'])].apply(pd.to_numeric, errors='coerce')


# Guiamets Preprocessing

In [233]:
guiamets_dfs.keys()

dict_keys(['cumulated_rainfall_24h', 'environmental_temperature'])

In [234]:
# Percentage of missing values
guiamets_dfs['cumulated_rainfall_24h'].isna().sum() / guiamets_dfs['cumulated_rainfall_24h'].shape[0]

Acumulado          0.000184
Fecha acumulado    0.000184
M�ximo             1.000000
Fecha m�ximo       1.000000
fecha              0.000000
dtype: float64

In [235]:
guiamets_dfs['environmental_temperature'].isna().sum() / guiamets_dfs['environmental_temperature'].shape[0]

M�ximo          0.013823
M�nimo          0.013823
Media           0.014005
Fecha m�ximo    0.013823
Fecha m�nimo    0.013823
fecha           0.000000
dtype: float64

In [236]:
guiamets_dfs['cumulated_rainfall_24h'].columns.to_list()

['Acumulado', 'Fecha acumulado', 'M�ximo', 'Fecha m�ximo', 'fecha']

In [237]:
guiamets_dfs['cumulated_rainfall_24h'].drop(columns=['Fecha m�ximo', 'M�ximo'], inplace=True)
guiamets_dfs['environmental_temperature'].drop(columns=['Fecha m�ximo', 'Fecha m�nimo'], inplace=True)

In [238]:
guiamets_dfs['cumulated_rainfall_24h'].rename(
    columns={
        'fecha': 'DateTime',
        'Acumulado': 'Average'
    },
    inplace=True
)

guiamets_dfs['environmental_temperature'].rename(
    columns={
        'fecha': 'DateTime',
        'Media': 'Average',
        'M�nimo': 'Minimum',
        'M�ximo': 'Maximum'
    },
    inplace=True
)

In [239]:
for df in guiamets_dfs.values():
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df[df.columns.difference(['DateTime'])] = df[df.columns.difference(['DateTime'])].apply(pd.to_numeric, errors='coerce')

# Mequinenza Preprocessing

In [240]:
mequinenza_dfs.keys()

dict_keys(['cumulated_rainfall_24h'])

In [241]:
mequinenza_dfs['cumulated_rainfall_24h'].isna().sum() / mequinenza_dfs['cumulated_rainfall_24h'].shape[0]

Acumulado          0.001175
Fecha acumulado    0.001175
M�ximo             1.000000
Fecha m�ximo       1.000000
fecha              0.000000
dtype: float64

In [242]:
mequinenza_dfs['cumulated_rainfall_24h'].drop(columns=['Fecha m�ximo', 'M�ximo', 'Fecha acumulado'], inplace=True)

In [243]:
mequinenza_dfs['cumulated_rainfall_24h'].rename(
    columns={
        'fecha': 'DateTime',
        'Acumulado': 'Average'
    },
    inplace=True
)

In [244]:
mequinenza_dfs['cumulated_rainfall_24h']['DateTime'] = pd.to_datetime(mequinenza_dfs['cumulated_rainfall_24h']['DateTime'])
mequinenza_dfs['cumulated_rainfall_24h'][mequinenza_dfs['cumulated_rainfall_24h'].columns.difference(['DateTime'])] = mequinenza_dfs['cumulated_rainfall_24h'][mequinenza_dfs['cumulated_rainfall_24h'].columns.difference(['DateTime'])].apply(pd.to_numeric, errors='coerce')

# Xerta Preprocessing

In [245]:
xerta_dfs.keys()

dict_keys(['nitrate', 'ABS254', 'pH', 'Ammonium', 'dissolvedoxygen', 'Conductivity', 'redoxpotential', 'turbidity', 'watertemperature'])

In [246]:
for feature, df in xerta_dfs.items():
    print('Feature:', feature)
    print()
    print('% missing values:')
    print()
    print(df.isna().sum() / df.shape[0])
    print()
    print('Column names:', df.columns.to_list())
    print()
    print('-' * 100)
    print()

Feature: nitrate

% missing values:

Fecha       0.0
Promedio    0.0
Mínimo      0.0
Máximo      0.0
Nºdat       0.0
NºOK        0.0
dtype: float64

Column names: ['Fecha', 'Promedio', 'Mínimo', 'Máximo', 'Nºdat', 'NºOK']

----------------------------------------------------------------------------------------------------

Feature: ABS254

% missing values:

Fecha       0.0
Promedio    0.0
Mínimo      0.0
Máximo      0.0
Nºdat       0.0
NºOK        0.0
dtype: float64

Column names: ['Fecha', 'Promedio', 'Mínimo', 'Máximo', 'Nºdat', 'NºOK']

----------------------------------------------------------------------------------------------------

Feature: pH

% missing values:

Fecha       0.0
Promedio    0.0
Mínimo      0.0
Máximo      0.0
Nºdat       0.0
NºOK        0.0
dtype: float64

Column names: ['Fecha', 'Promedio', 'Mínimo', 'Máximo', 'Nºdat', 'NºOK']

----------------------------------------------------------------------------------------------------

Feature: Ammonium

% missing va

In [247]:
for df in xerta_dfs.values():
    df.rename(
        columns={
            'Fecha': 'DateTime',
            'Promedio': 'Average',
            'Máximo': 'Maximum',
            'Mínimo': 'Minimum'
        },
        inplace=True
    )
    
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df[df.columns.difference(['DateTime'])] = df[df.columns.difference(['DateTime'])].apply(pd.to_numeric, errors='coerce')

# Save Data

In [248]:
datasets_dict['TORTOSA'] = tortosa_dfs
datasets_dict['GUIAMETS'] = guiamets_dfs
datasets_dict['MEQUINENZA'] = mequinenza_dfs
datasets_dict['XERTA'] = xerta_dfs

In [249]:
for location in datasets_dict.keys():
    if not os.path.exists(os.path.join(intermediate_data_folder, location)):
        os.makedirs(os.path.join(intermediate_data_folder, location))
        
    path = os.path.join(intermediate_data_folder, location)
    for feature, df in datasets_dict[location].items():
        df.to_excel(os.path.join(path, f'{feature}.xlsx'), index=False)